In [1]:
import bw2data as bd
import bw2calc as bc
from time import time
from scipy import sparse
import numpy as np

09:28:01+0200 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


In [2]:
bd.projects.set_current("Speed racer 🏎️")

# 1. Basic approach re-using the factorization

In [3]:
db = bd.Database("ecoinvent-3.11-cutoff")

nodes = [node for node in db]
nodes.sort(key=lambda x:x.id)
fus = nodes[:250]

m = (
    'ecoinvent-3.11',
    'ReCiPe 2016 v1.03, endpoint (H) no LT',
    'ecosystem quality no LT',
    'photochemical oxidant formation: terrestrial ecosystems no LT'
)

demand, data_objs, _ = bd.prepare_lca_inputs(demand={fus[0]: 1}, method=m, remapping=False)

lca = bc.LCA(demand, data_objs=data_objs)
lca.lci()
lca.lcia()

start = time()

for node in fus:
    lca.lci(demand={node.id: 1})
    lca.lcia_calculation()

print(f"{len(fus)} nodes and 1 impact category took {(time() - start) / len(fus)} seconds per calculation")

250 nodes and 1 impact category took 0.006463779449462891 seconds per calculation


Profiling results: `multiple-full-calcs.html`

# 2. We don't need to diagonalize the demand array if we only want scores

We can compress the inventory to sum all elementary flows

In [4]:
start = time()

for node in fus:
    lca.build_demand_array({node.id: 1})
    supply_array = lca.solve_linear_system()
    supply_array_one_column = sparse.coo_matrix(
        (supply_array, (np.arange(supply_array.shape[0]), np.zeros(supply_array.shape[0]))),
        shape=(len(lca.dicts.activity), 1),
    )
    inventory_matrix_one_column = lca.biosphere_matrix @ supply_array_one_column
    score = (lca.characterization_matrix @ inventory_matrix_one_column).sum()

print(f"{len(fus)} nodes and 1 impact category took {(time() - start) / len(fus)} seconds per calculation")

250 nodes and 1 impact category took 0.003922901153564453 seconds per calculation


Profiling results: `multiple-calcs-only-score.html`

# 3. We can multiply the characterization matrix by the biosphere once, in advance

In [5]:
start = time()

characterized_inventory = lca.characterization_matrix @ lca.biosphere_matrix

for node in fus:
    lca.build_demand_array({node.id: 1})
    supply_array = lca.solve_linear_system()
    supply_array_one_column = sparse.coo_matrix(
        (supply_array, (np.arange(supply_array.shape[0]), np.zeros(supply_array.shape[0]))),
        shape=(len(lca.dicts.activity), 1),
    )
    score = (characterized_inventory @ supply_array_one_column).sum()

print(f"{len(fus)} nodes and 1 impact category took {(time() - start) / len(fus)} seconds per calculation")

250 nodes and 1 impact category took 0.0017997913360595704 seconds per calculation


Profiling results: `pre-characterize.html`

# 4. If we know the technosphere hasn't changed, we can skip that check from `pypardiso`

In [6]:
from pypardiso.pardiso_wrapper import PyPardisoSolver

start = time()

solver = PyPardisoSolver()
solver.factorize(lca.technosphere_matrix)

characterized_inventory = lca.characterization_matrix @ lca.biosphere_matrix

for node in fus:
    lca.build_demand_array({node.id: 1})

    b = solver._check_b(lca.technosphere_matrix, lca.demand_array)
    solver.set_phase(33)
    supply_array = solver._call_pardiso(lca.technosphere_matrix, b)

    supply_array_one_column = sparse.coo_matrix(
        (supply_array, (np.arange(supply_array.shape[0]), np.zeros(supply_array.shape[0]))),
        shape=(len(lca.dicts.activity), 1),
    )
    score = (characterized_inventory @ supply_array_one_column).sum()

print(f"{len(fus)} nodes and 1 impact category took {(time() - start) / len(fus)} seconds per calculation")

250 nodes and 1 impact category took 0.002495290756225586 seconds per calculation


Profiling results: `skip-matrix-checks.html`

# 5. We can push more than one functional unit at a time to the sparse solver

In [7]:
fus = nodes[:2000]

start = time()

solver = PyPardisoSolver()
solver.factorize(lca.technosphere_matrix)

characterized_inventory = lca.characterization_matrix @ lca.biosphere_matrix

for i in range(0, len(fus), 100):
    i = 0
    chunk = fus[i:i + 100]

    demand_array = np.zeros((len(lca.dicts.product), len(chunk)))

    for i, node in enumerate(chunk):
        demand_array[lca.dicts.product[node.id], i] = 1

    b = solver._check_b(lca.technosphere_matrix, demand_array)
    solver.set_phase(33)
    supply_array = solver._call_pardiso(lca.technosphere_matrix, b)
    supply_matrix = sparse.csc_matrix(supply_array)
    score = (characterized_inventory @ supply_matrix).sum(axis=0)

print(f"{len(fus)} nodes and 1 impact category took {(time() - start) / len(fus)} seconds per calculation")

2000 nodes and 1 impact category took 0.0008670732975006103 seconds per calculation


Profiling results: `chunked.html`

# 6. Constructing sparse matrices isn't needed when we have dense vectors. Let's keep things in numpy.

In [9]:
start = time()

solver = PyPardisoSolver()
solver.factorize(lca.technosphere_matrix)

characterized_inventory = np.asarray((lca.characterization_matrix @ lca.biosphere_matrix).sum(axis=0))

def calculate_score(x, y):
    return x @ y

for i in range(0, len(fus), 100):
    i = 0
    chunk = fus[i:i + 100]

    demand_array = np.zeros((len(lca.dicts.product), len(chunk)))

    for i, node in enumerate(chunk):
        demand_array[lca.dicts.product[node.id], i] = 1

    b = solver._check_b(lca.technosphere_matrix, demand_array)
    solver.set_phase(33)
    supply_array = solver._call_pardiso(lca.technosphere_matrix, b)
    calculate_score(characterized_inventory, supply_array)

print(f"{len(fus)} nodes and 1 impact category took {(time() - start) / len(fus)} seconds per calculation")

2000 nodes and 1 impact category took 0.0009325989484786988 seconds per calculation


OK, maybe that was a step too far. We have the best approach implemented in `bw2calc` version 2.2. Let's see how long a full calculation takes:

In [10]:
from bw2calc.fast_scores import FastScoresOnlyMultiLCA

In [11]:
fus = {str(node.id): {node.id: 1} for node in db}
len(fus)

25412

In [12]:
method_config = {"impact_categories": [
    ic 
    for ic in bd.methods 
    if ic[0] == 'ecoinvent-3.11'
    and 'EN15804' not in str(ic)
]}
len(method_config["impact_categories"])

592

In [13]:
fsmlca = FastScoresOnlyMultiLCA(
    demands=fus,
    method_config=method_config,
    data_objs=bd.get_multilca_data_objs(fus, method_config)
)

In [14]:
start = time()

fsmlca.calculate()

end = time()

print(f"{len(fus)} nodes and {len(method_config["impact_categories"])} impact category took {(end - start) / len(fus)} seconds per functional unit.\nTotal time was {end - start} seconds")

25412 nodes and 592 impact category took 0.000836735942242522 seconds per functional unit.
Total time was 21.263133764266968 seconds
